In [1]:
from tensorflow.keras import layers

In [4]:
layers.Average?

Init signature: layers.Average(*args, **kwargs)
Docstring:     
Layer that averages a list of inputs element-wise.

It takes as input a list of tensors, all of the same shape, and returns
a single tensor (also of the same shape).

Example:

>>> x1 = np.ones((2, 2))
>>> x2 = np.zeros((2, 2))
>>> y = tf.keras.layers.Average()([x1, x2])
>>> y.numpy().tolist()
[[0.5, 0.5], [0.5, 0.5]]

Usage in a functional model:

>>> input1 = tf.keras.layers.Input(shape=(16,))
>>> x1 = tf.keras.layers.Dense(8, activation='relu')(input1)
>>> input2 = tf.keras.layers.Input(shape=(32,))
>>> x2 = tf.keras.layers.Dense(8, activation='relu')(input2)
>>> avg = tf.keras.layers.Average()([x1, x2])
>>> out = tf.keras.layers.Dense(4)(avg)
>>> model = tf.keras.models.Model(inputs=[input1, input2], outputs=out)

Raises:
  ValueError: If there is a shape mismatch between the inputs and the shapes
    cannot be broadcasted to match.
Init docstring:
Intializes a Merge layer.

Arguments:
  **kwargs: standard layer keywor

In [3]:
dir(layers)

['AbstractRNNCell',
 'Activation',
 'ActivityRegularization',
 'Add',
 'AdditiveAttention',
 'AlphaDropout',
 'Attention',
 'Average',
 'AveragePooling1D',
 'AveragePooling2D',
 'AveragePooling3D',
 'AvgPool1D',
 'AvgPool2D',
 'AvgPool3D',
 'BatchNormalization',
 'Bidirectional',
 'Concatenate',
 'Conv1D',
 'Conv2D',
 'Conv2DTranspose',
 'Conv3D',
 'Conv3DTranspose',
 'ConvLSTM2D',
 'Convolution1D',
 'Convolution2D',
 'Convolution2DTranspose',
 'Convolution3D',
 'Convolution3DTranspose',
 'Cropping1D',
 'Cropping2D',
 'Cropping3D',
 'Dense',
 'DenseFeatures',
 'DepthwiseConv2D',
 'Dot',
 'Dropout',
 'ELU',
 'Embedding',
 'Flatten',
 'GRU',
 'GRUCell',
 'GaussianDropout',
 'GaussianNoise',
 'GlobalAveragePooling1D',
 'GlobalAveragePooling2D',
 'GlobalAveragePooling3D',
 'GlobalAvgPool1D',
 'GlobalAvgPool2D',
 'GlobalAvgPool3D',
 'GlobalMaxPool1D',
 'GlobalMaxPool2D',
 'GlobalMaxPool3D',
 'GlobalMaxPooling1D',
 'GlobalMaxPooling2D',
 'GlobalMaxPooling3D',
 'Input',
 'InputLayer',
 'Input

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

import tensorflow as tf
import tensorflow.keras.backend as ktf
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.models import load_model,Model
from tensorflow.keras.preprocessing import image

from location.network import *
from location.losses import *
from recognition.network import *

from util import *
import cfg


In [ ]:
img_path="demo/046.jpg"
img = image.load_img(img_path)
d_wight, d_height = resize_image(img, cfg.image_size)

plt.imshow(img)

In [ ]:
image.img_to_array(img)
image.random_brightness(img,brightness_range=(0.8,0.8))
plt.imshow(img)

In [ ]:
#img = img.resize((d_wight, d_height), Image.NEAREST).convert('RGB')
img = image.img_to_array(img)
#img = img/255*2-1
preprocess_input(img,mode='tf')
#plt.imshow(image.array_to_img(img))
#image.random_brightness(img,brightness_range=(0.5,0.9))
plt.imshow(img)
plt.imshow(image.array_to_img(img))
#image.img_to_array(img)


In [ ]:
plt.imshow(image.array_to_img(img))

# 模型载入

In [ ]:
location_model = tf.keras.models.load_model(r"location\saved_model\location_pb",compile=False)
location_model.compile(loss=quad_loss,optimizer='Nadam')
tf.keras.models.save_model(location_model ,r"location\saved_model\location_model.h5")

In [ ]:
location_model = Location().location_network()
location_model.load_weights(cfg.location_weights)
location_model.summary()

In [ ]:
_, recognition_model = CRNN(cfg.width, cfg.height, cfg.label_len, cfg.characters).network()
recognition_model.load_weights(cfg.recognition_weights)
recognition_model.summary()

## 单图测试

In [ ]:
%%time
img_path = 'demo/046.jpg'
img = image.load_img(img_path,target_size=(512,512))
plt.figure(figsize=(15,20))
plt.subplot(1,3,1)
plt.xlabel('input')
plt.imshow(img)
ims_re = location(location_model, img_path,cfg.pixel_threshold)
if(len(ims_re)>0):
    #print(len(ims_re))
    plt.subplot(1,3,2)
    plt.imshow(ims_re[0])
    re_text = recognition(recognition_model, ims_re[0])
    result = img_path + " " + re_text + "\n"
    print('recognize result: '+ re_text)
        

## 检测计时

In [ ]:
%%time
img_path = 'demo/046.jpg'
img = image.load_img(img_path,target_size=(512,512))
plt.figure(figsize=(15,20))
plt.subplot(1,3,1)
plt.xlabel('input')
plt.imshow(img)
ims_re = location(location_model, img_path,cfg.pixel_threshold)
if(len(ims_re)>0):
    #print(len(ims_re))
    plt.subplot(1,3,2)
    plt.imshow(ims_re[0])

## 识别计时

In [ ]:
%%time
re_text = recognition(recognition_model, ims_re[0])
result = img_path + " " + re_text + "\n"
print(result)

## 文件夹图片检测识别

In [ ]:
%%time
imgs = os.listdir('demo')
for img in imgs:
    img_path = os.path.join('demo', img)
    ims_re = location(location_model, img_path, cfg.pixel_threshold)
    if len(ims_re) > 0:
        for i in range(len(ims_re)):
            re_text = recognition(recognition_model, ims_re[0])
            result = img + " " + re_text + "\n"
            print(result)

# 全数据集测试

In [ ]:
labels = 'labels.txt'
def acc_calculate(labels='labels.txt',results='result.txt'):
    with open(labels) as label:
        linexs = label.readlines()
    with open(results) as result:
        lineys = result.readlines()
    cnt = 0
    for linex in linexs:
        linex = linex.split()
        for liney in lineys:
            liney = liney.split()
            if linex[0] == liney[0] and len(liney) > 1:
                if linex[1] == liney[1]:
                    cnt = cnt + 1
    acc=cnt/len(lineys)
    return  acc

In [ ]:
acc=acc_calculate(labels,"recognition/result.txt")
print('{0:.3%}'.format(acc))

# 检测测试

## 训练集

In [ ]:
%%time
results='east_train_result.txt'
with open(results, 'w+') as txt:
    with open(r'Z:\Code\Python\datas\Augment_meter\east_train.txt') as val:
        lines =val.readlines()
        for line in lines:
            img_name=line.split(',')[0]
            img_path=os.path.join('Z:\Code\Python\datas\Augment_meter\imgs_east_512',img_name)
            ims_re = location(location_model, img_path, cfg.pixel_threshold)
            if len(ims_re) > 0:
                re_text = recognition(recognition_model, ims_re[0])
                result =  img_name+ " " + re_text + "\n"
                #print(result)
                txt.write(result)

In [ ]:
acc=acc_calculate(labels,results)
print('{0:.3%}'.format(acc))

## 验证集

In [ ]:
%%time
results='east_val_result.txt'
with open(results, 'w+') as txt:
    with open(r'Z:\Code\Python\datas\Augment_meter\east_val.txt') as val:
        lines =val.readlines()
        for line in lines:
            img_name=line.split(',')[0]
            img_path=os.path.join('Z:\Code\Python\datas\Augment_meter\imgs_east_512',img_name)
            ims_re = location(location_model, img_path, cfg.pixel_threshold)
            if len(ims_re) > 0:
                for i in range(len(ims_re)):
                    re_text = recognition(recognition_model, ims_re[i])
                    result =  img_name+ " " + re_text + "\n"
                    print(result)
                    txt.write(result)

In [ ]:
acc=acc_calculate(labels,results)
print('{0:.3%}'.format(acc))

# 识别测试

## 训练集

In [ ]:
%%time
results='crnn_train_result.txt'
with open(results, 'w+') as txt:
    with open(r'Z:\Code\Python\datas\Augment_meter\crnn_train.txt') as val:
        lines =val.readlines()
        for line in lines:
            img_name=line.split(' ')[0]
            img_path=os.path.join('Z:\Code\Python\datas\Augment_meter\imgs_east_512',img_name)
            ims_re = location(location_model, img_path, cfg.pixel_threshold)
            if len(ims_re) > 0:
                re_text = recognition(recognition_model, ims_re[0])
                result =  img_name+ " " + re_text + "\n"
                print(result)
                txt.write(result)

In [ ]:
acc=acc_calculate(labels,results)
print('{0:.3%}'.format(acc))

## 验证集

In [ ]:
%%time
results='crnn_val_result.txt'
with open(results, 'w+') as txt:
    with open(r'Z:\Code\Python\datas\Augment_meter\crnn_val.txt') as val:
        lines =val.readlines()
        for line in lines:
            img_name=line.split(' ')[0]
            img_path=os.path.join('Z:\Code\Python\datas\Augment_meter\imgs_east_512',img_name)
            ims_re = location(location_model, img_path, cfg.pixel_threshold)
            if len(ims_re) > 0:
                for i in range(len(ims_re)):
                    re_text = recognition(recognition_model, ims_re[i])
                    result =  img_name+ " " + re_text + "\n"
                    print(result)
                    txt.write(result)

In [ ]:
acc=acc_calculate(labels,results)
print('{0:.3%}'.format(acc))